# Продвинутый Pandas

**Задание 1**
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

In [2]:
log = pd.read_csv('visit_log.csv', sep = ';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
log.loc[log.traffic_source.isin(['yandex', 'google']), 'source_type'] = 'organic'

In [4]:
log.loc[(log.traffic_source.isin(['paid', 'email'])) & (log.region == 'Russia'), 'source_type'] = 'ad'

In [5]:
log.loc[(log.traffic_source.isin(['paid', 'email'])) & (log.region != 'Russia'), 'source_type'] = 'other'

In [6]:
log.source_type = log.source_type.fillna(log.traffic_source)

In [7]:
log.source_type.value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

In [8]:
log.pivot_table(index='source_type', columns = 'traffic_source', values='timestamp', aggfunc='count', fill_value = 0, margins=True)

traffic_source,direct,email,google,paid,yandex,All
source_type,,,,,,
ad,0,2209,0,1581,0,3790
direct,7013,0,0,0,0,7013
organic,0,0,2507,0,2900,5407
other,0,1666,0,1062,0,2728
All,7013,3875,2507,2643,2900,18938


**Задание 2**
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц 
с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию:
    /, затем 8 цифр, затем дефис. Выполните следующие действия:
1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение 
в соответствии с заданным шаблоном

In [9]:
import re

In [10]:
urls = pd.read_csv('URLs.txt')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [11]:
urls[urls.url.str.contains(r'/\d{8}-', regex=True)].tail(10)

,url
83,/world/36215106-lider-ig-obratilsya-k-boevikam...
84,/articles/36066499-iz-za-zvaniy-anny-netrebko-...
85,/starlife/36096316-u-lyudmily-maksakovoy-sdali...
86,/economics/36097033-a-zhit-za-schet-rossii-vy-...
87,/starlife/36096182-vdova-valeriya-zolotuhina-v...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...
92,/world/36103095-dominikanskih-zhurnalistov-ubi...


**Задание 3**
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. 
Леммы должны иметь строковый тип.

In [12]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [13]:
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [14]:
from pymystem3 import Mystem

In [15]:
def lemmas(row):
    m = Mystem()
    lemmas = m.lemmatize(row.keyword.lower())
    l = []
    for i in lemmas:
        if (i.isalpha()) & (len(i)>1):
            l.append(i)
    text = ' '.join(l)
    return text

In [16]:
data['lemmas'] = data.apply(lemmas, axis=1)

In [17]:
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна рубль
1,доллары в рубли,114173,доллар рубль
2,100 долларов в рублях,97534,доллар рубль
3,курс рубля,53546,курс рубль
